In [2]:
import pandas as pd
import json
import pyarrow

# Nota: Se debe especificar la ruta del archivo Parquet
parquet_archivo = 'ruta/del/archivo/business.parquet'

# Cargar el archivo Parquet a un dataframe
df = pd.read_parquet(parquet_archivo, engine='pyarrow')

# Muestra aleatoria de 5 filas del DataFrame 
df.sample(5)

,ID_CLIENTE,ANTIGUEDAD_MESES,TIPO_TARJETA,LINEA_CREDITO,RNG_LINEA,DEUDA_TARJETA,SEGMENTO_BANCO,AMBITO,ZONA,EDAD,SEXO,ECIVIL,Total_facturacion,Num_compras
1103,C0002169,R1. <0-6 M>,CLASICA,800,2000,507.15,SEGMENTO BRONCE,L,CENTRO,32.0,F,SOLTERO,693.86,9
170,C0000314,R4. [24-MAS>,CLASICA,3200,4000,1679.95,SEGMENTO PLATA,L,NORTE,59.0,F,SOLTERO,1535.40,10
24,C0000035,R4. [24-MAS>,CLASICA,9900,10000,41.66,SEGMENTO ORO,L,MODERNA,NaN,F,CASADO,291.58,9
778,C0001445,R3. [12-24 M>,ORO,3000,4000,2977.70,SEGMENTO PLATA,L,NORTE,31.0,M,SOLTERO,477.00,13
770,C0001429,R4. [24-MAS>,CLASICA,1800,2000,715.50,SEGMENTO PLATA,L,MODERNA,40.0,M,CASADO,1249.20,13


In [3]:
#Mostrar resumen útil de la estructura de los datos después de cargarlos con los nombres de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID_CLIENTE         1220 non-null   object 
 1   ANTIGUEDAD_MESES   1220 non-null   object 
 2   TIPO_TARJETA       1220 non-null   object 
 3   LINEA_CREDITO      1220 non-null   int64  
 4   RNG_LINEA          1220 non-null   int64  
 5   DEUDA_TARJETA      1220 non-null   float64
 6   SEGMENTO_BANCO     1220 non-null   object 
 7   AMBITO             1220 non-null   object 
 8   ZONA               1220 non-null   object 
 9   EDAD               1034 non-null   float64
 10  SEXO               1220 non-null   object 
 11  ECIVIL             1220 non-null   object 
 12  Total_facturacion  1220 non-null   float64
 13  Num_compras        1220 non-null   int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 133.6+ KB


In [5]:
#  Seleccionar columnas necesarias
columnas_necesarias = [
    'ID_CLIENTE', 'ANTIGUEDAD_MESES', 'TIPO_TARJETA', 
    'LINEA_CREDITO', 'RNG_LINEA', 'DEUDA_TARJETA', 
    'SEGMENTO_BANCO', 'AMBITO', 'ZONA', 'EDAD', 
    'SEXO', 'ECIVIL', 'Total_facturacion', 'Num_compras'
]

if all(col in df.columns for col in columnas_necesarias):
    df_transformado = df[columnas_necesarias]
else:
    print(f'Las columnas necesarias {columnas_necesarias} no están presentes en el archivo Parquet.')
    df_transformado = df

# Agrupar por ID_CLIENTE excluyendo la columna de agrupación en el resultado de apply
agrupar_data = df_transformado.groupby('ID_CLIENTE', group_keys=False).apply(lambda x: x.to_dict(orient='records'), include_groups=False).to_dict()

# Exportar a JSON con una estructura coherente
#Nota: se debe ingresar su ruta para guardas el archivo .json
json_salida = 'ruta/del/archivo/business2.json'
with open(json_salida, 'w', encoding='utf-8') as file:
    json.dump(agrupar_data, file, indent=4, ensure_ascii=False)

print(f'Data transformada y agrupada exportada a {json_salida}')


Data transformada y agrupada exportada a C:/Users/evely/Desktop/business2.json


In [6]:
# Ahora se puede mostrar el archivo exportado a Json 

#Leer el archivo JSON para verificar la estructura
with open(json_salida, 'r', encoding='utf-8') as file:
    df_json = json.load(file)

# Imprimir ID_CLIENTE y registros asociados 
for cliente_id, registros in df_json.items():
    print(f"ID_CLIENTE: {cliente_id}")
    for registro in registros:
        registro_sin_id = {k: v for k, v in registro.items() if k != 'ID_CLIENTE'}
        print(json.dumps(registro_sin_id, indent=4, ensure_ascii=False))
    print("\n")  # Añadir una línea en blanco entre clientes para mayor claridad

# Mostrar algunas filas del DataFrame original
print(df_transformado.head())

# Mostrar información general del DataFrame original
print(df_transformado.info())




ID_CLIENTE: C0000001
{
    "ANTIGUEDAD_MESES": "R4. [24-MAS>",
    "TIPO_TARJETA": "CLASICA",
    "LINEA_CREDITO": 30000,
    "RNG_LINEA": 30000,
    "DEUDA_TARJETA": 0.0,
    "SEGMENTO_BANCO": "SEGMENTO BRONCE",
    "AMBITO": "L",
    "ZONA": "CALLAO",
    "EDAD": 65.0,
    "SEXO": "M",
    "ECIVIL": "CASADO",
    "Total_facturacion": 478.8,
    "Num_compras": 1
}


ID_CLIENTE: C0000002
{
    "ANTIGUEDAD_MESES": "R4. [24-MAS>",
    "TIPO_TARJETA": "CLASICA",
    "LINEA_CREDITO": 21000,
    "RNG_LINEA": 22000,
    "DEUDA_TARJETA": 0.0,
    "SEGMENTO_BANCO": "SEGMENTO ORO",
    "AMBITO": "L",
    "ZONA": "MODERNA",
    "EDAD": 67.0,
    "SEXO": "F",
    "ECIVIL": "SOLTERO",
    "Total_facturacion": 1374.15,
    "Num_compras": 3
}


ID_CLIENTE: C0000004
{
    "ANTIGUEDAD_MESES": "R4. [24-MAS>",
    "TIPO_TARJETA": "CLASICA",
    "LINEA_CREDITO": 31500,
    "RNG_LINEA": 32000,
    "DEUDA_TARJETA": 115.16,
    "SEGMENTO_BANCO": "SEGMENTO ORO",
    "AMBITO": "L",
    "ZONA": "MODERNA",
    